In [270]:
import os
import pandas as pd
import pickle
import numpy as np
import re
import seaborn as sns
import matplotlib.pyplot as plt
pd.set_option('display.max_colwidth', 1000)

In [271]:
filename = 'list_of_dicts.pkl'
with open(filename, 'rb') as file:
    list_of_dicts = pickle.load(file) 

In [272]:

all_relationship_terms = [
    'Spouses',
    'Spouse',
    'Spouse(s)',
    'Partners',
    'Partner',
    'Partner(s)',
]

spouse_terms = [
    'Spouses',
    'Spouse',
    'Spouse(s)',
]

children_terms = [
    'Children',
    'Parent',
    'Parents',
    'Mother',
    'Father',
    'Parent(s)',
    'Children',
]

In [304]:
tmp_df = pd.DataFrame(list_of_dicts)
for entry in list_of_dicts:
    if entry['actor'] == 'Michael Cera':
        print(entry)

{'actor': 'Michael Cera', 'url': 'https://en.wikipedia.org/wiki/Michael_Cera', 'Born': 'Michael Austin Cera ( 1988-06-07 ) June 7, 1988 (age\xa036) Brampton , Ontario , Canada', 'Occupations': 'Actor comedian musician', 'Years\xa0active': '1999–present', 'Spouse': 'Nadine', 'Children': '2', 'Genres': 'Lo-fi indie folk', 'Instruments': 'Vocals bass', 'Website': 'michaelceramusic .bandcamp .com'}


# Create DataFrame

In [298]:
filtered_data = [
    {
        k: v.replace('\u200b', '') if k in all_relationship_terms else v
        for k, v in entry.items() if k in ['actor'] + all_relationship_terms
    }
    for entry in list_of_dicts
] # add this to replace \xa0

# Define the regex pattern to match each marriage/partnership
pattern = re.compile(r'([a-zA-Z\s]+\(.*?\))')

# Initialize an empty list to store the rows
rows = []

# Iterate through each entry in the filtered_data
for entry in filtered_data:
    actor = entry['actor']
    for key in all_relationship_terms:
        if key in entry:
            # Find all matches for the current relationship term (Spouses, Partners, etc.)
            matches = pattern.findall(entry[key])
            for match in matches:
                rows.append({
                    'actor': actor,
                    'variable': key,
                    'value': match.strip()  # Strip any leading/trailing whitespace
                })
    if not any(x in all_relationship_terms for x in entry.keys()):
        rows.append({
                'actor': actor,
                'variable': 'Spouses',
                'value': 'never'  # Strip any leading/trailing whitespace
            })



In [306]:
# Create the DataFrame from the rows
df = pd.DataFrame(rows, columns=['actor', 'variable', 'value'])
len(tmp_df) - df.actor.nunique()

tmp_df[~tmp_df['actor'].isin(df.actor.unique())][['actor'] + all_relationship_terms]

,actor,Spouses,Spouse,Spouse(s),Partners,Partner,Partner(s)
90,Chadwick Boseman,NaN,Taylor Simone Ledward,NaN,NaN,NaN,NaN
594,Elizabeth Olsen,NaN,Robbie Arnett,NaN,NaN,NaN,NaN
673,Romain Duris,NaN,NaN,NaN,NaN,Olivia Bonamy,NaN
676,Bérénice Bejo,NaN,Michel Hazanavicius,NaN,NaN,NaN,NaN
688,Sophie Okonedo,NaN,Jamie Chalmers,NaN,NaN,NaN,NaN
705,Barry Pepper,NaN,Cindy Pepper,NaN,NaN,NaN,NaN
726,Léa Seydoux,NaN,NaN,NaN,NaN,André Meyer,NaN
823,Elijah Wood,NaN,NaN,NaN,NaN,Mette-Marie Kongsved,NaN
928,Mark Strong,NaN,Liza Marshall,NaN,NaN,NaN,NaN
950,Sung Kang,NaN,Miki Yim,NaN,NaN,NaN,NaN


In [289]:
## extract names & the details
df[['name', 'details']] = df['value'].str.extract(r'([a-zA-Z\s]+)\s*(\(.*?\))').apply(lambda x: x.str.strip())
pattern = re.compile(r'([a-zA-Z]+\.? \d{4})\s* *;? *\s*([a-zA-Z]+\.? \d{4})?') #re.compile(r'([a-zA-Z]{1,4}\.\s*\d{4})(?:;\s*([a-zA-Z]{1,4}\.\s*\d{4}))?')

## Apply the regex to extract the matches into two columns
df[['first_event', 'second_event']] = df['details'].str.extract(pattern)

## get out the m. and the year
df['first_abbrev'] = df['first_event'].str.extract(r'(\w+)\.? ')
df['second_abbrev'] = df['second_event'].str.extract(r'(\w+)\.? ')
## if your marriage makes it, then give the np.nan a "surv" for survived
df.loc[ (df.variable.isin(spouse_terms)) & ( df['second_abbrev'].isnull() ), 'second_abbrev'] = 'surv'



In [290]:
df

,actor,variable,value,name,details,first_event,second_event,first_abbrev,second_abbrev
0,Robert De Niro,Spouses,Diahnne Abbott ( m. 1976; div. 1988),Diahnne Abbott,( m. 1976; div. 1988),m. 1976,div. 1988,m,div
1,Robert De Niro,Spouses,Grace Hightower ( m. 1997; sep. 2018),Grace Hightower,( m. 1997; sep. 2018),m. 1997,sep. 2018,m,sep
2,Robert De Niro,Spouses,never,NaN,NaN,NaN,NaN,NaN,surv
3,Robert De Niro,Spouses,never,NaN,NaN,NaN,NaN,NaN,surv
4,Robert De Niro,Spouses,never,NaN,NaN,NaN,NaN,NaN,surv
...,...,...,...,...,...,...,...,...,...
6697,Jerry Lewis,Spouses,never,NaN,NaN,NaN,NaN,NaN,surv
6698,Jerry Lewis,Spouses,never,NaN,NaN,NaN,NaN,NaN,surv
6699,Jerry Lewis,Spouses,never,NaN,NaN,NaN,NaN,NaN,surv
6700,Jerry Lewis,Spouses,never,NaN,NaN,NaN,NaN,NaN,surv


# QA + Sanity Checking

In [ ]:
## check how many don't have semi-colons in them
len(df[df['variable'].isin(spouse_terms)])
df[df['variable'].isin(spouse_terms)].isnull().sum()
## can add a check to see how many have two sets of numbers indicating two events

df[(df['variable'].isin(spouse_terms)) & (df['first_event'].notnull())]['first_event'].unique()

# come back and address
df[(df['variable'].isin(spouse_terms)) & (df['first_event'].isnull())][['value', 'first_event']]
df[(df['variable'].isin(spouse_terms)) & (df['second_event'].isnull())][['value', 'second_event']]
df[(df['variable'].isin(spouse_terms)) & (df['second_event'].isnull())]['value'].unique()

## looking at abbrevs
df['first_abbrev'].unique()
len(df[ ( df['variable'].isin(spouse_terms) ) & ~( df['first_abbrev'].isin(['m']) ) ]) # not too bad

df['second_abbrev'].unique()
df[ ( df['variable'].isin(spouse_terms) ) & ( df['second_abbrev'].isnull())  ].details.unique()


# Analysis

In [ ]:
working = df[df['variable'].isin(spouse_terms)]

## make bind annulments, separations, and divorces together. died and survived are the same
working = working.replace('ann', 'ended').replace('annul', 'ended').replace('div', 'ended').replace('sep', 'ended')\
                 .replace('died', 'survived').replace('surv', 'survived').replace('m', 'married')

## drop the broken ones
working = working[~working['first_abbrev'].isin(['ended','c'])]


##### Counts of marriages, actors
## Number of actors who were married
working.actor.nunique()

## Number of marriages. 
len(working)

## count of marriages per actor
actor_marriage_count_table = working.groupby('actor')['first_abbrev'].count().reset_index().rename(columns = {'first_abbrev':'count'})

## Num of actors who were married 1,2,3, etc. times
marriage_distro_table = actor_marriage_count_table.groupby('count')['actor'].count().reset_index()
cutoff_number = 3
## create 3+ group
marriage_distro_table['group'] = marriage_distro_table['count'].apply(lambda x: f'{cutoff_number}+' if x >= cutoff_number else str(x))
# Group by the new 'group' column and sum the 'actor' counts
marriage_distro_trunc = marriage_distro_table.groupby('group', as_index=False)['actor'].sum()
marriage_distro_trunc

In [ ]:
# Barchart function
def basic_barchart(x, y, df, xlabel, ylabel, title, hue = None):
    if hue:
        ax = sns.barplot(x=x, y=y, data=df, hue=hue, edgecolor='black')
    else:
        ax = sns.barplot(x=x, y=y, data=df, edgecolor='black')

    # for index, row in df.iterrows():
    #     # addition = row['count'] * 0.02
    #     ax.text(row[x], index + 10, row['actor'], color='black', ha="center", va="bottom")
    # Add text labels
    for container in ax.containers:
        # Determine whether the data is in percentage format (decimals) or whole numbers
        labels = []
        for v in container.datavalues:
            if isinstance(v, float) and v < 1:
                labels.append(f'{v:.0%}')  # Convert to percentage format
            else:
                labels.append(f'{int(v)}')  # Keep as a whole number

        ax.bar_label(container, labels=labels, label_type='edge', padding=3)


    # Adding labels and title
    plt.xlabel(xlabel)
    plt.ylabel(ylabel)
    plt.title(title)

    # Show the plot
    plt.show()

In [ ]:
x = 'count'
y = 'actor'
df = marriage_distro_table
xlabel = 'Number of Marriages'
ylabel = 'Number of Actors'
title = 'Number of Actors Who Have Been Married N Times'
basic_barchart(x, y, df, xlabel, ylabel, title)

In [ ]:
x = 'group'
y = 'actor'
df = marriage_distro_trunc
xlabel = 'Number of Marriages'
ylabel = 'Number of Actors'
title = 'Number of Actors Who Have Been Married N Times'
basic_barchart(x, y, df, xlabel, ylabel, title)

In [ ]:
plt.clf()
plt.figure(figsize=(8, 8))
plt.pie(marriage_distro_trunc['actor'], labels=marriage_distro_trunc['group'], autopct='%1.1f%%', startangle=140, colors=sns.color_palette('pastel'))

# Add a title
plt.title('Percent of Actors Who Have Been Married N Times')

# Display the pie chart
plt.show()

In [ ]:
##### Rates of Divorces

assert working['actor'].isnull().sum() == 0

## divorce rate across all
divorce_counts = working.groupby(['first_abbrev', 'second_abbrev'])['actor'].count()
divorce_counts / divorce_counts.sum()# lost 40ish marriages, can look into those


## divorce rates for 1st, 2nd, 3rd etc. marriages

"""
what we can do is based on what number marriage it is, label it in the main table. We could do this with a window function, if you show up 3 times in the main table you had 3 marriages, 2 of which died.
Then groupby the 1st, 2nd, 3rd and the surv/ended counts
""" 
working['marriage_number'] = working.groupby(['actor']).cumcount() + 1
working.loc[working['marriage_number'] >= cutoff_number, 'marriage_number_trunc'] = f'{cutoff_number}+'
working.loc[working['marriage_number'] < cutoff_number, 'marriage_number_trunc'] = working.loc[working['marriage_number'] < cutoff_number, 'marriage_number']
# working['marriage_number'] = 'marriage no. ' + working['marriage_number'].apply(str)

working.groupby(['marriage_number', 'second_abbrev'])['actor'].count() / working.groupby('marriage_number')['actor'].count()#.sum()
working.groupby(['marriage_number_trunc', 'second_abbrev'])['actor'].count() / working.groupby('marriage_number_trunc')['actor'].count()#.sum()


In [ ]:
# divorce_counts.reset_index()
working.groupby(['marriage_number_trunc', 'second_abbrev'])['actor'].count().reset_index()
# working.groupby(['marriage_number_trunc', 'second_abbrev'])['actor'].count()

In [ ]:
x = 'marriage_number_trunc'
y = 'actor'
hue = 'second_abbrev'
df = working.groupby(['marriage_number_trunc', 'second_abbrev'])['actor'].count().reset_index()
xlabel = 'Marriage Number'
ylabel = 'Number of Actors'
title = 'Outcomes of 1st, 2nd, 3rd+ Marriages'
basic_barchart(x, y, df, xlabel, ylabel, title, hue)

In [ ]:
working.groupby(['marriage_number_trunc', 'second_abbrev'])['actor'].count().reset_index()
# working.groupby(['marriage_number_trunc', 'second_abbrev'])['actor'].count()

x = 'marriage_number_trunc'
y = 'actor'
hue = 'second_abbrev'
df = (working.groupby(['marriage_number_trunc', 'second_abbrev'])['actor'].count() / working.groupby(['marriage_number_trunc'])['actor'].count()).reset_index()
xlabel = 'Number of Marriages'
ylabel = 'Number of Actors'
title = 'Survival Rates of 1st, 2nd, 3rd+ Marriages'
basic_barchart(x, y, df, xlabel, ylabel, title, hue)

In [ ]:
x = 'marriage_number'
y = 'actor'
hue = 'second_abbrev'
df = (working.groupby(['marriage_number', 'second_abbrev'])['actor'].count() / working.groupby(['marriage_number'])['actor'].count()).reset_index()
xlabel = 'Number of Marriages'
ylabel = 'Number of Actors'
title = 'Survival Rates of 1st, 2nd, N Marriages'
basic_barchart(x, y, df, xlabel, ylabel, title, hue)

# Additional QA

In [ ]:
working[ ( working['second_event'].isnull() ) & ( working['second_abbrev'].isnull() ) ] # this check, need to make sure that there's no second events we're missing
working[working['first_abbrev'] == 'c']
working[working['first_abbrev'] == 'ended']
df[df['actor'] == 'Jean Dujardin']
actor_marriage_count_table[actor_marriage_count_table['count'] == 0]

In [ ]:
working[working['actor'] == 'Christoph Waltz']